In [1]:
import pandas as pd
import numpy as np

In [2]:
TRAIN_FILE_PATH = "booking_train_set.csv" #"/content/drive/MyDrive/booking_wisdom/booking_train_set.csv" #"booking_train_set.csv"
# LIST_FILE_PATH =  "train_predictions_no_last_city.csv.gzip"

LIST_FILE_PATH =  "train_test_no_test_last_city.csv.gzip"

TEST_FILE_PATH = "booking_test_set.csv" #"/content/drive/MyDrive/booking_wisdom/booking_train_set.csv" #"booking_train_set.csv"
# TEST_LIST_FILE_PATH =  "list_booking_train.csv.gz"#"/content/drive/MyDrive/booking_wisdom/list_booking_train.csv.gz" #"list_booking_train.csv.gz"


In [3]:
dtypes_dict = {
#     "utrip_id":"category",
#     "city_id":"int32", ## will impute nans as 0 if used
    "rank":"int16",
    "label": "int8",#"uint8",
    "affiliate_id":"int16",
#     "device_class":"category",
#     "booker_country":"string", ## category would save more memory but intereferes with feature engineering
#     "hotel_country":"string"
}

In [4]:
## list of candidate predictions (100 per utrip_id)
df = pd.read_csv(LIST_FILE_PATH,
                      compression="gzip",
                        usecols=[1,2,3], # skip city_id col
#                        nrows=1000,
#                         dtype=dtypes_dict
                     )

df.rename(columns={"user_id":"utrip_id"},inplace=True)
df["rank"] = df.groupby("utrip_id").transform("cumcount")

# print("rank of labels (target =1):")
# print(df_list.loc[df_list["label"]==1]["rank"].describe(percentiles=[.25,.5,.75,.9,.95,.99]))


# df_list.set_index("utrip_id",inplace=True)

list_ids_all = list(set(df["utrip_id"]))
print(len(list_ids_all))
display(df)

288348


,utrip_id,city_id,score,rank
0,727105_1,23921,4.622820,0
1,727105_1,29319,4.383900,1
2,727105_1,48483,4.361497,2
3,727105_1,5325,4.072368,3
4,727105_1,0,4.038220,4
...,...,...,...,...
43252195,3072980_1,60056,1.264239,145
43252196,3072980_1,18036,1.246247,146
43252197,3072980_1,2334,1.242764,147
43252198,3072980_1,44770,1.242372,148


In [5]:
df_train = pd.read_csv(TRAIN_FILE_PATH,usecols=["utrip_id","city_id","checkin"],parse_dates=["checkin"],infer_datetime_format=True)
df_train.sort_values([ "checkin",
                "utrip_id"],inplace=True)

df_train["row_num"] = df_train.groupby("utrip_id")["checkin"].rank(ascending=True,pct=False).astype(int)
utrip_counts = df_train["utrip_id"].value_counts()
df_train["total_rows"] = df_train["utrip_id"].map(utrip_counts)


df_train = df_train.loc[df_train.row_num == df_train.total_rows]
df_train = df_train[["city_id","utrip_id"]]

train_ids = list(set(df_train["utrip_id"]))

df_train["label"] = 1

##### test data - currenbtly not covered with preds

In [6]:
df_test = pd.read_csv(TEST_FILE_PATH,usecols=["utrip_id","city_id","checkin","row_num","total_rows"],parse_dates=["checkin"],infer_datetime_format=True)

df_test.sort_values([  "utrip_id","row_num","checkin",],inplace=True)
display(df_test)

,checkin,utrip_id,row_num,total_rows,city_id
0,2016-07-21,1000066_2,1,4,56430
1,2016-07-23,1000066_2,2,4,41971
2,2016-07-25,1000066_2,3,4,5797
3,2016-07-28,1000066_2,4,4,0
4,2016-02-08,1000270_1,1,4,50075
...,...,...,...,...,...
378662,2016-10-07,999911_1,5,5,0
378663,2016-08-15,999991_3,1,4,29770
378664,2016-08-18,999991_3,2,4,36170
378665,2016-08-19,999991_3,3,4,52155


In [7]:
df_test.describe()

,row_num,total_rows,city_id
count,378667.000000,378667.000000,378667.00000
mean,3.541695,6.083390,27193.51751
std,2.328596,2.695393,21928.96179
min,1.000000,4.000000,0.00000
25%,2.000000,4.000000,4975.00000
50%,3.000000,5.000000,25173.00000
75%,5.000000,7.000000,47389.00000
max,44.000000,44.000000,67563.00000


In [8]:
df_test["utrip_id"].value_counts().describe()

count    70662.000000
mean         5.358849
std          1.970472
min          4.000000
25%          4.000000
50%          5.000000
75%          6.000000
max         44.000000
Name: utrip_id, dtype: float64

In [9]:
df_test = df_test.loc[df_test.row_num == df_test.total_rows]
df_test = df_test[["city_id","utrip_id"]].drop_duplicates()

print("unique utrip ids in test",df_test["utrip_id"].nunique())

unique utrip ids in test 70662


In [10]:
df_test.loc[df_test["utrip_id"].isin(list_ids_all)]["utrip_id"].nunique()

70662

In [11]:
assert df_test.loc[df_test["utrip_id"].isin(list_ids_all)]["utrip_id"].nunique() == df_test["utrip_id"].nunique()

## merge - train only for now, later: test

In [12]:
df

,utrip_id,city_id,score,rank
0,727105_1,23921,4.622820,0
1,727105_1,29319,4.383900,1
2,727105_1,48483,4.361497,2
3,727105_1,5325,4.072368,3
4,727105_1,0,4.038220,4
...,...,...,...,...
43252195,3072980_1,60056,1.264239,145
43252196,3072980_1,18036,1.246247,146
43252197,3072980_1,2334,1.242764,147
43252198,3072980_1,44770,1.242372,148


In [13]:
# ###df_train = pd.concat([df_train,df_test])

In [14]:
## add labels, for train data only. Still keep test candidates (without labels)
df = df.merge(df_train,on=["utrip_id","city_id"],how="left")
df["label"] = df["label"].fillna(0).astype(np.int8)
print(df["rank"].isna().sum())
print(df.shape)
print(df["utrip_id"].nunique())
display(df)

0
(43252200, 5)
288348


,utrip_id,city_id,score,rank,label
0,727105_1,23921,4.622820,0,0
1,727105_1,29319,4.383900,1,0
2,727105_1,48483,4.361497,2,0
3,727105_1,5325,4.072368,3,0
4,727105_1,0,4.038220,4,0
...,...,...,...,...,...
43252195,3072980_1,60056,1.264239,145,0
43252196,3072980_1,18036,1.246247,146,0
43252197,3072980_1,2334,1.242764,147,0
43252198,3072980_1,44770,1.242372,148,0


In [15]:
print(df["rank"].isna().sum())

0


#### keep only train ids

In [16]:
# print(df.shape[0])
# df = df.loc[df["utrip_id"].isin(train_ids)]
print(df.shape[0])

43252200


In [17]:
df["utrip_id"].nunique()

288348

In [18]:
df.sort_values(["utrip_id","label","rank"],inplace=True,ascending=[False,False,True])

In [19]:
df

,utrip_id,city_id,score,rank,label
25811250,999991_3,0,3.756113,0,0
25811251,999991_3,29770,3.128314,1,0
25811252,999991_3,36063,3.007151,2,0
25811253,999991_3,17127,2.863316,3,0
25811254,999991_3,64876,2.765818,4,0
...,...,...,...,...,...
25173745,1000027_1,38497,1.946920,145,0
25173746,1000027_1,42368,1.945058,146,0
25173747,1000027_1,19740,1.944693,147,0
25173748,1000027_1,48898,1.935032,148,0


In [20]:
df.to_csv("predictions_list_proc_v2.csv.gz",index=False,compression="gzip")